# 🔧 Preprocesamiento de Datos
## Clasificación de Niveles de Obesidad - Regresión Ordinal

Este notebook realiza el preprocesamiento completo de los datos antes de entrenar los modelos.

**Objetivos del preprocesamiento:**
1. Codificar variables categóricas (convertir texto a números)
2. Normalizar/estandarizar variables numéricas
3. Dividir datos en Train (70%) y Test (30%) con estratificación
4. Guardar datos preprocesados y transformadores

**Distribución**: 70% Train (con validación cruzada) / 30% Test

---

## ¿Por qué es necesario el preprocesamiento?

Los algoritmos de Machine Learning requieren que los datos estén en un formato específico:

- **Variables categóricas**: Deben convertirse a números (encoding)
- **Escalas diferentes**: Variables con rangos muy diferentes (ej: Age 0-100 vs Height 1.5-2.0) pueden sesgar el modelo
- **División de datos**: Necesitamos separar datos para entrenar y evaluar

Sin preprocesamiento adecuado, los modelos pueden tener mal rendimiento o incluso fallar.


## 1. Importación de Librerías

### ¿Por qué estas librerías?

- **pandas**: Manipulación de datos estructurados (DataFrames)
- **numpy**: Operaciones numéricas y matemáticas
- **sklearn**: Proporciona herramientas de preprocesamiento (StandardScaler, LabelEncoder, etc.) y división de datos
- **pickle**: Guardar objetos Python (transformadores) para uso futuro
- **os**: Crear directorios si no existen

### ¿Por qué guardar los transformadores?

Es crucial guardar los transformadores (scaler, encoders) porque:
- Cuando tengamos nuevos datos, debemos aplicar las **mismas transformaciones**
- El scaler debe usar los mismos parámetros (media y desviación estándar) aprendidos del train
- Sin esto, los nuevos datos estarían en una escala diferente y las predicciones serían incorrectas


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import os
import warnings

warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")


✓ Librerías importadas correctamente


## 2. Carga del Dataset

### ¿Qué hace este paso?

Carga el archivo CSV original y verifica que:
- El archivo se carga correctamente
- No hay valores faltantes
- La estructura es la esperada

### ¿Por qué verificar valores faltantes?

Los valores faltantes pueden causar problemas:
- Algunos algoritmos no pueden manejar NaN directamente
- Necesitamos decidir cómo tratarlos (eliminar, imputar, etc.)
- En este caso, verificamos que no haya valores faltantes antes de continuar


In [2]:
# Cargar el dataset
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

print(f"✓ Dataset cargado exitosamente")
print(f"  - Forma del dataset: {df.shape}")
print(f"  - Columnas: {list(df.columns)}")
print(f"\nPrimeras 3 filas:")
df.head(3)


✓ Dataset cargado exitosamente
  - Forma del dataset: (2111, 17)
  - Columnas: ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad']

Primeras 3 filas:


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


In [3]:
# Verificar valores faltantes
print(f"Valores faltantes por columna:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "  ✓ No hay valores faltantes")


Valores faltantes por columna:
  ✓ No hay valores faltantes


## 3. Creación de Variable IMC (Índice de Masa Corporal)

### ¿Qué hace este paso?

Calcula el IMC (BMI) a partir de Weight y Height, y elimina las variables originales.

### ¿Por qué usar IMC en lugar de Weight y Height?

**IMC (Índice de Masa Corporal)** = Peso (kg) / Altura (m)²

**Ventajas**:
1. **Relevancia clínica**: El IMC es la métrica estándar para clasificar obesidad
2. **Reducción de dimensionalidad**: 2 variables (Weight, Height) → 1 variable (BMI)
3. **Reduce multicolinealidad**: Weight y Height están altamente correlacionadas
4. **Interpretabilidad**: El IMC es más interpretable y significativo
5. **Normalización natural**: El IMC ya normaliza por altura

**Las clases de obesidad están basadas en rangos de IMC**:
- Insufficient_Weight: IMC < 18.5
- Normal_Weight: 18.5 ≤ IMC < 25
- Overweight_Level_I: 25 ≤ IMC < 27
- Overweight_Level_II: 27 ≤ IMC < 30
- Obesity_Type_I: 30 ≤ IMC < 35
- Obesity_Type_II: 35 ≤ IMC < 40
- Obesity_Type_III: IMC ≥ 40

### ¿Por qué eliminar Weight y Height?

- El IMC captura la información relevante de ambas variables
- Evita multicolinealidad (correlación alta entre variables)
- Simplifica el modelo sin perder información relevante para este problema


In [5]:
# Calcular IMC (BMI) = Weight (kg) / Height (m)²
print("📊 Calculando IMC (Índice de Masa Corporal)...")
df['BMI'] = df['Weight'] / (df['Height'] ** 2)

print(f"✓ IMC calculado")
print(f"  - Rango de IMC: {df['BMI'].min():.2f} - {df['BMI'].max():.2f}")
print(f"  - Media de IMC: {df['BMI'].mean():.2f}")

# Mostrar algunos ejemplos antes de eliminar
print(f"\nEjemplos de cálculo:")
print(df[['Height', 'Weight', 'BMI', 'NObeyesdad']].head())

# Eliminar Weight y Height (ya no las necesitamos)
print(f"\n📝 Eliminando variables Weight y Height...")
df = df.drop(columns=['Weight', 'Height'])
print(f"  ✓ Variables eliminadas")
print(f"  - Columnas restantes: {len(df.columns)}")

# Variable objetivo
target_column = 'NObeyesdad'
y = df[target_column].copy()

# Características (todas las columnas excepto la objetivo)
X = df.drop(columns=[target_column])

print(f"\n✓ Variable objetivo separada: '{target_column}'")
print(f"  - Forma de y: {y.shape}")
print(f"  - Forma de X: {X.shape}")

# Identificar tipos de variables
print(f"\n📊 ANÁLISIS DE TIPOS DE VARIABLES:")
print("-" * 80)

# Variables numéricas (ya son números)
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nVariables Numéricas ({len(numeric_cols)}):")
for col in numeric_cols:
    print(f"  - {col}: min={X[col].min():.2f}, max={X[col].max():.2f}, media={X[col].mean():.2f}")

# Variables categóricas (objetos/strings)
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"\nVariables Categóricas ({len(categorical_cols)}):")
for col in categorical_cols:
    unique_vals = X[col].unique()
    print(f"  - {col}: {len(unique_vals)} valores únicos -> {list(unique_vals)[:5]}...")


📊 Calculando IMC (Índice de Masa Corporal)...


KeyError: 'Weight'

## 4. Encoding de Variables Categóricas

### ¿Qué hace este paso?

Convierte variables categóricas (texto) a números. Los algoritmos de ML solo pueden trabajar con números.

### ¿Por qué necesitamos encoding?

Los modelos matemáticos no entienden texto. Necesitamos convertir:
- "Male" / "Female" → números
- "yes" / "no" → números
- "Public_Transportation" / "Walking" / etc. → números

### Dos Estrategias de Encoding

#### 1. Label Encoding
- Convierte cada categoría a un número (0, 1, 2, ...)
- Ejemplo: "no" → 0, "yes" → 1
- **Usamos para variables binarias** (solo 2 valores)

#### 2. One-Hot Encoding
- Crea una columna binaria por cada categoría
- Ejemplo: "Public_Transportation" → [1, 0, 0, 0, 0]
- **Usamos para variables con múltiples categorías sin orden**
- Evita que el modelo asuma orden donde no lo hay

### ¿Por qué diferentes estrategias?

- **Label Encoding para binarias**: Más eficiente, no crea columnas extra
- **One-Hot para multi-categoría**: Evita que el modelo piense que hay orden (ej: "Walking" no es "menor" que "Automobile")


In [7]:
# Crear una copia para trabajar
X_encoded = X.copy()

# Identificar variables binarias (solo 2 valores)
binary_cols = []
multi_cat_cols = []

for col in categorical_cols:
    n_unique = X[col].nunique()
    if n_unique == 2:
        binary_cols.append(col)
    else:
        multi_cat_cols.append(col)

print(f"Variables Binarias (Label Encoding): {binary_cols}")
print(f"Variables Multi-Categoría (One-Hot Encoding): {multi_cat_cols}")


Variables Binarias (Label Encoding): ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
Variables Multi-Categoría (One-Hot Encoding): ['CAEC', 'CALC', 'MTRANS']


In [8]:
# 4.1: Label Encoding para variables binarias
print(f"\n📝 Aplicando Label Encoding a variables binarias...")
label_encoders = {}

for col in binary_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col])
    label_encoders[col] = le
    print(f"  ✓ {col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")



📝 Aplicando Label Encoding a variables binarias...
  ✓ Gender: {'Female': 0, 'Male': 1}
  ✓ family_history_with_overweight: {'no': 0, 'yes': 1}
  ✓ FAVC: {'no': 0, 'yes': 1}
  ✓ SMOKE: {'no': 0, 'yes': 1}
  ✓ SCC: {'no': 0, 'yes': 1}


In [9]:
# 4.2: One-Hot Encoding para variables multi-categoría
print(f"\n📝 Aplicando One-Hot Encoding a variables multi-categoría...")

# Usar pandas get_dummies (más simple que sklearn para este caso)
X_encoded = pd.get_dummies(X_encoded, columns=multi_cat_cols, prefix=multi_cat_cols, drop_first=False)

print(f"  ✓ Variables codificadas:")
print(f"    - Antes: {len(categorical_cols)} variables categóricas")
print(f"    - Después: {X_encoded.shape[1]} variables totales")
print(f"    - Nuevas columnas creadas: {X_encoded.shape[1] - len(numeric_cols) - len(binary_cols)}")

# Mostrar algunas columnas nuevas
new_cols = [col for col in X_encoded.columns if any(mc in col for mc in multi_cat_cols)]
print(f"\n  Ejemplo de nuevas columnas (primeras 5):")
for col in new_cols[:5]:
    print(f"    - {col}")



📝 Aplicando One-Hot Encoding a variables multi-categoría...
  ✓ Variables codificadas:
    - Antes: 8 variables categóricas
    - Después: 25 variables totales
    - Nuevas columnas creadas: 13

  Ejemplo de nuevas columnas (primeras 5):
    - CAEC_Always
    - CAEC_Frequently
    - CAEC_Sometimes
    - CAEC_no
    - CALC_Always


## 5. Normalización/Estandarización de Variables Numéricas

### ¿Qué hace este paso?

Normaliza/estandariza las variables numéricas para que todas estén en la misma escala.

### ¿Por qué es importante?

**Problema**: Variables con rangos muy diferentes pueden dominar el modelo.

**Ejemplo**:
- Age: 0-100 (rango grande)
- Height: 1.5-2.0 (rango pequeño)

Sin estandarización, Age podría tener más "peso" en el modelo simplemente por tener números más grandes, aunque Height podría ser igual de importante.

### Tipos de Normalización

#### Estandarización (Z-score normalization) - **USAMOS ESTA**
- **Fórmula**: `z = (x - μ) / σ`
- Convierte datos a: media = 0, desviación estándar = 1
- Útil cuando los datos siguen distribución normal
- **Ventaja**: Funciona bien con la mayoría de algoritmos

#### Normalización (Min-Max)
- **Fórmula**: `x_norm = (x - min) / (max - min)`
- Convierte datos a rango [0, 1]
- Útil cuando no conocemos la distribución

### ¿Cuándo NO estandarizar?

- **Árboles de decisión** (Random Forest, Gradient Boosting): No necesitan estandarización porque dividen por umbrales
- **Naive Bayes**: Puede funcionar sin estandarización
- **Pero**: SVM, k-NN, redes neuronales SÍ necesitan estandarización

**Para este proyecto**: Estandarizamos porque usaremos varios tipos de modelos.


In [12]:
# Identificar columnas numéricas (después del encoding)
# Las columnas numéricas originales + las binarias codificadas
numeric_cols_to_scale = numeric_cols + binary_cols

print(f"Variables a estandarizar ({len(numeric_cols_to_scale)}):")
print(f"  {numeric_cols_to_scale}")

# Crear el scaler (pero NO aplicarlo aún - lo haremos después de dividir)
# Esto es importante: NO debemos estandarizar antes de dividir train/test
# porque podríamos "filtrar" información del test al train

scaler = StandardScaler()

print(f"\n✓ Scaler creado (se aplicará después de dividir train/test)")
print(f"  - Tipo: StandardScaler (Z-score normalization)")
print(f"  - Fórmula: z = (x - μ) / σ")

# Mostrar estadísticas antes de estandarizar
print(f"\n📊 Estadísticas ANTES de estandarizar (primeras 3 variables):")
for col in numeric_cols_to_scale[:3]:
    print(f"  {col}:")
    print(f"    - Media: {X_encoded[col].mean():.4f}")
    print(f"    - Desv. Est.: {X_encoded[col].std():.4f}")
    print(f"    - Min: {X_encoded[col].min():.4f}")
    print(f"    - Max: {X_encoded[col].max():.4f}")


Variables a estandarizar (12):
  ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'BMI', 'Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']

✓ Scaler creado (se aplicará después de dividir train/test)
  - Tipo: StandardScaler (Z-score normalization)
  - Fórmula: z = (x - μ) / σ

📊 Estadísticas ANTES de estandarizar (primeras 3 variables):
  Age:
    - Media: 24.3126
    - Desv. Est.: 6.3460
    - Min: 14.0000
    - Max: 61.0000
  FCVC:
    - Media: 2.4190
    - Desv. Est.: 0.5339
    - Min: 1.0000
    - Max: 3.0000
  NCP:
    - Media: 2.6856
    - Desv. Est.: 0.7780
    - Min: 1.0000
    - Max: 4.0000


## 6. División de Datos (70-30) con Estratificación

### ¿Qué hace este paso?

Divide los datos en Train (70%) y Test (30%) manteniendo la proporción de clases en ambos conjuntos.

### ¿Por qué dividir los datos?

- **Train (70%)**: Usamos para entrenar y ajustar los modelos (con validación cruzada)
- **Test (30%)**: Usamos SOLO al final para evaluar el modelo final
- **Nunca** usamos test durante el entrenamiento (evita sobreajuste)

### ¿Qué es la Estratificación?

**Estratificación** = Mantener la proporción de clases en ambos conjuntos.

**Ejemplo sin estratificación**:
- Train: 80% Normal_Weight, 5% Obesity_Type_III
- Test: 10% Normal_Weight, 30% Obesity_Type_III
- ❌ Problema: Los conjuntos no son representativos

**Ejemplo con estratificación**:
- Train: 13.6% Normal_Weight, 15.3% Obesity_Type_III
- Test: 13.6% Normal_Weight, 15.3% Obesity_Type_III
- ✅ Ambos conjuntos tienen la misma distribución

### ¿Por qué NO estandarizar antes de dividir?

**IMPORTANTE**: NO estandarizamos antes de dividir porque:

1. El scaler se ajusta SOLO con datos de train
2. Luego se aplica a test (sin reajustar)
3. Esto evita **"data leakage"** (filtrar información del test al train)

Si estandarizamos antes de dividir:
- El scaler "ve" todos los datos (train + test)
- Esto filtra información del test al train
- El modelo podría tener mejor rendimiento de lo que realmente tiene


In [13]:
# División estratificada
# random_state: Para reproducibilidad (mismos resultados cada vez)
# stratify: Mantiene proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.30,  # 30% para test
    random_state=42,  # Semilla para reproducibilidad
    stratify=y  # Estratificación por clases
)

print(f"\n✓ División completada:")
print(f"  - Train: {X_train.shape[0]} registros ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Test: {X_test.shape[0]} registros ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"  - Características: {X_train.shape[1]}")



✓ División completada:
  - Train: 1477 registros (70.0%)
  - Test: 634 registros (30.0%)
  - Características: 25


In [14]:
# Verificar estratificación
print(f"\n📊 Verificación de estratificación:")
print(f"\nDistribución en Train:")
train_dist = y_train.value_counts().sort_index()
train_pct = (y_train.value_counts(normalize=True) * 100).sort_index()
for clase in train_dist.index:
    print(f"  {clase:25s}: {train_dist[clase]:4d} ({train_pct[clase]:5.2f}%)")

print(f"\nDistribución en Test:")
test_dist = y_test.value_counts().sort_index()
test_pct = (y_test.value_counts(normalize=True) * 100).sort_index()
for clase in test_dist.index:
    print(f"  {clase:25s}: {test_dist[clase]:4d} ({test_pct[clase]:5.2f}%)")

# Verificar que las proporciones son similares
print(f"\n✓ Verificación: Las proporciones son similares en ambos conjuntos")



📊 Verificación de estratificación:

Distribución en Train:
  Insufficient_Weight      :  190 (12.86%)
  Normal_Weight            :  201 (13.61%)
  Obesity_Type_I           :  245 (16.59%)
  Obesity_Type_II          :  208 (14.08%)
  Obesity_Type_III         :  227 (15.37%)
  Overweight_Level_I       :  203 (13.74%)
  Overweight_Level_II      :  203 (13.74%)

Distribución en Test:
  Insufficient_Weight      :   82 (12.93%)
  Normal_Weight            :   86 (13.56%)
  Obesity_Type_I           :  106 (16.72%)
  Obesity_Type_II          :   89 (14.04%)
  Obesity_Type_III         :   97 (15.30%)
  Overweight_Level_I       :   87 (13.72%)
  Overweight_Level_II      :   87 (13.72%)

✓ Verificación: Las proporciones son similares en ambos conjuntos


## 7. Estandarización de Datos (DESPUÉS de Dividir)

### ¿Qué hace este paso?

Aplica la estandarización SOLO a los datos de train, ajusta el scaler con train, y luego aplica la misma transformación a test (sin reajustar).

### ¿Por qué este orden es crítico?

**Proceso correcto**:
1. Dividir datos (train/test)
2. Ajustar scaler con SOLO train → aprende media y desviación estándar de train
3. Transformar train con el scaler ajustado
4. Transformar test con el MISMO scaler (sin reajustar)

**Si hiciéramos mal**:
1. Estandarizar todo el dataset
2. Dividir datos
3. ❌ El scaler "vio" datos de test → data leakage

### ¿Por qué no reajustar el scaler con test?

Porque en producción:
- Nuevos datos llegan sin etiquetas
- Debemos aplicar las mismas transformaciones aprendidas del train
- Si reajustamos con test, estaríamos "haciendo trampa"

### Resultado Esperado

Después de estandarizar:
- **Media ≈ 0**: Los datos están centrados
- **Desviación estándar ≈ 1**: Los datos están escalados
- Esto facilita el entrenamiento de modelos sensibles a la escala


In [19]:
# Ajustar scaler SOLO con datos de train
print(f"\n📝 Ajustando scaler con datos de train...")
scaler.fit(X_train[numeric_cols_to_scale])

# Aplicar transformación
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_cols_to_scale] = scaler.transform(X_train[numeric_cols_to_scale])
X_test_scaled[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])

print(f"✓ Estandarización aplicada")

# Mostrar estadísticas después de estandarizar
print(f"\n📊 Estadísticas DESPUÉS de estandarizar (primeras 3 variables):")
for col in numeric_cols_to_scale[:3]:
    print(f"  {col}:")
    print(f"    - Media: {X_train_scaled[col].mean():.6f} (debe ser ~0)")
    print(f"    - Desv. Est.: {X_train_scaled[col].std():.6f} (debe ser ~1)")
    print(f"    - Min: {X_train_scaled[col].min():.4f}")
    print(f"    - Max: {X_train_scaled[col].max():.4f}")



📝 Ajustando scaler con datos de train...
✓ Estandarización aplicada

📊 Estadísticas DESPUÉS de estandarizar (primeras 3 variables):
  Age:
    - Media: 0.000000 (debe ser ~0)
    - Desv. Est.: 1.000339 (debe ser ~1)
    - Min: -1.3202
    - Max: 5.8305
  FCVC:
    - Media: -0.000000 (debe ser ~0)
    - Desv. Est.: 1.000339 (debe ser ~1)
    - Min: -2.7065
    - Max: 1.0833
  NCP:
    - Media: -0.000000 (debe ser ~0)
    - Desv. Est.: 1.000339 (debe ser ~1)
    - Min: -2.1733
    - Max: 1.6959


## 8. Encoding de Variable Objetivo (Ordinal)

### ¿Qué hace este paso?

Convierte las clases de texto a números ordinales manteniendo el orden.

### ¿Por qué encoding ordinal?

El problema es **regresión ordinal**: las clases tienen un orden natural:
1. Insufficient_Weight (menor peso)
2. Normal_Weight
3. Overweight_Level_I
4. Overweight_Level_II
5. Obesity_Type_I
6. Obesity_Type_II
7. Obesity_Type_III (mayor peso)

### Mapeo Ordinal

```
0: Insufficient_Weight
1: Normal_Weight
2: Overweight_Level_I
3: Overweight_Level_II
4: Obesity_Type_I
5: Obesity_Type_II
6: Obesity_Type_III
```

### ¿Por qué es importante mantener el orden?

- Algunos modelos pueden aprovechar el orden (regresión ordinal)
- Las métricas ordinales consideran la distancia en la escala
- Clasificar Obesity_Type_I como Obesity_Type_III es peor que clasificarlo como Normal_Weight


In [20]:
# Definir orden ordinal
ordinal_order = [
    'Insufficient_Weight',
    'Normal_Weight',
    'Overweight_Level_I',
    'Overweight_Level_II',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III'
]

# Crear encoder ordinal
target_encoder = LabelEncoder()
target_encoder.fit(ordinal_order)

# Aplicar encoding
y_train_encoded = pd.Series(target_encoder.transform(y_train), index=y_train.index)
y_test_encoded = pd.Series(target_encoder.transform(y_test), index=y_test.index)

print(f"\n✓ Variable objetivo codificada:")
print(f"  Mapeo de clases:")
for i, clase in enumerate(ordinal_order):
    print(f"    {i}: {clase}")

print(f"\n  Distribución en Train (codificada):")
print(y_train_encoded.value_counts().sort_index())

print(f"\n  Distribución en Test (codificada):")
print(y_test_encoded.value_counts().sort_index())



✓ Variable objetivo codificada:
  Mapeo de clases:
    0: Insufficient_Weight
    1: Normal_Weight
    2: Overweight_Level_I
    3: Overweight_Level_II
    4: Obesity_Type_I
    5: Obesity_Type_II
    6: Obesity_Type_III

  Distribución en Train (codificada):
0    190
1    201
2    245
3    208
4    227
5    203
6    203
Name: count, dtype: int64

  Distribución en Test (codificada):
0     82
1     86
2    106
3     89
4     97
5     87
6     87
Name: count, dtype: int64


## 9. Guardado de Datos Preprocesados y Transformadores

### ¿Qué hace este paso?

Guarda los datos preprocesados y los transformadores para uso futuro.

### ¿Por qué guardar los datos preprocesados?

- **Reproducibilidad**: Podemos cargar los datos ya procesados sin ejecutar todo el notebook
- **Eficiencia**: No necesitamos preprocesar cada vez
- **Consistencia**: Aseguramos usar exactamente los mismos datos

### ¿Por qué guardar los transformadores?

**CRÍTICO**: Cuando tengamos nuevos datos para predecir:
1. Debemos aplicar las **mismas transformaciones**
2. El scaler debe usar los mismos parámetros (media y desviación estándar del train)
3. Los encoders deben usar el mismo mapeo

**Sin esto**: Los nuevos datos estarían en una escala diferente y las predicciones serían incorrectas.

### Archivos que Guardamos

- **Datos preprocesados**: X_train.csv, X_test.csv, y_train.csv, y_test.csv
- **Transformadores**: scaler.pkl, target_encoder.pkl, label_encoders.pkl
- **Información**: preprocessing_info.pkl (metadatos sobre el preprocesamiento)


In [21]:
# Crear directorio si no existe
os.makedirs('data/processed', exist_ok=True)
os.makedirs('models/preprocessing', exist_ok=True)

# Guardar datos preprocesados
print(f"\n📁 Guardando datos preprocesados...")

# Guardar como CSV (para inspección)
X_train_scaled.to_csv('data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('data/processed/X_test.csv', index=False)
y_train_encoded.to_csv('data/processed/y_train.csv', index=False)
y_test_encoded.to_csv('data/processed/y_test.csv', index=False)

# Guardar también las versiones originales (sin codificar) para referencia
y_train.to_csv('data/processed/y_train_original.csv', index=False)
y_test.to_csv('data/processed/y_test_original.csv', index=False)

print(f"  ✓ Datos guardados en 'data/processed/'")



📁 Guardando datos preprocesados...
  ✓ Datos guardados en 'data/processed/'


In [22]:
# Guardar transformadores
print(f"\n📁 Guardando transformadores...")

with open('models/preprocessing/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('models/preprocessing/target_encoder.pkl', 'wb') as f:
    pickle.dump(target_encoder, f)

with open('models/preprocessing/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

# Guardar información sobre las columnas
preprocessing_info = {
    'numeric_cols': numeric_cols,
    'binary_cols': binary_cols,
    'multi_cat_cols': multi_cat_cols,
    'numeric_cols_to_scale': numeric_cols_to_scale,
    'ordinal_order': ordinal_order,
    'n_features': X_train_scaled.shape[1]
}

with open('models/preprocessing/preprocessing_info.pkl', 'wb') as f:
    pickle.dump(preprocessing_info, f)

print(f"  ✓ Transformadores guardados en 'models/preprocessing/'")



📁 Guardando transformadores...
  ✓ Transformadores guardados en 'models/preprocessing/'


## 10. Resumen Final

### ✅ Lo que hemos completado:

1. **Carga y exploración** del dataset original
2. **Identificación** de tipos de variables (numéricas vs categóricas)
3. **Encoding** de variables categóricas (Label + One-Hot)
4. **Estandarización** de variables numéricas
5. **División estratificada** 70-30 (Train/Test)
6. **Encoding ordinal** de variable objetivo
7. **Guardado** de datos preprocesados y transformadores

### 📊 Resultados:

- **Dataset original**: 2111 registros, 17 columnas
- **Características finales**: 26 variables (después de encoding)
- **Train**: 1477 registros (70%)
- **Test**: 634 registros (30%)

### 🚀 Próximos Pasos:

1. Los datos están listos para entrenar modelos
2. Usar validación cruzada en el conjunto de train
3. Evaluar en el conjunto de test al final

### ⚠️ Recordatorios Importantes:

- **Nunca** usar el conjunto de test durante el entrenamiento
- **Siempre** aplicar las mismas transformaciones a nuevos datos
- **Guardar** los transformadores para producción

---

**Nota**: Este preprocesamiento es fundamental. Cualquier error aquí afectará todos los modelos que entrenemos después.
